In [21]:
import torch
import torch.nn as nn
kernel_List = [12, 4, 4, 4, 4, 4]
channel_List = [128, 256, 512, 512, 512, 400]

class ConvBlock(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_sz, padding, stride = 2) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_sz, stride, padding)
        self.bn = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        x = self.dropout(x)
        return x

def get_convBlocks(in_channel):
    layerNum = len(kernel_List)
    blocks = []
    blocks.append(ConvBlock(in_channel, channel_List[0], kernel_List[0], int(kernel_List[0] / 2 - 1)))
    for i in range(1, layerNum):
        blocks.append(ConvBlock(channel_List[i-1], channel_List[i], kernel_List[i], int(kernel_List[i] / 2 - 1)))
    return blocks

class DeepFold(nn.Module):
    def __init__(self, in_channel) -> None:
        super().__init__()
        self.convLayer = nn.Sequential(*get_convBlocks(in_channel))
    
    # [batch_size, 3, 256, 256]
    def forward(self, x):
        # [batch_size, 400, 4, 4]
        x = self.convLayer(x)
        # [batch_size, 400, 4]
        x = torch.diagonal(x, dim1=2, dim2=3)
        # [batch_size, 400]
        x = torch.mean(x, dim= 2)

        normValue = torch.norm(x, dim = 1) # norm_value [batch_size]
        # print(normValue.shape)
        # [400, batch_size]  除法要求最后一维要和norm_value维度匹配
        x = x.permute(1, 0)
        # [400, batch_size] 已经正则化
        x = torch.div(x, normValue)

        # [batch_size, 400]
        x = x.permute(1, 0)
        return x

    # def hook(self, layer: nn.Module, input: torch.tensor, output)

# outputList = []
# def hook(self, layer: nn.Module,  output: torch.tensor):
#     outputList.append(output)

x = torch.rand(2, 3, 256, 256)

model = DeepFold(3)

# for layer in model.convLayer:
#     layer.register_forward_hook(hook)

y = model(x)

print(torch.norm(y, dim = 1).shape)

# for ele in outputList:
#     print(ele.shape)

# print(model)
# print(help(model))
# print(len(list(model.named_modules())))
# for name,_ in model.convLayer.named_modules():
#     print(name)
#     print('-'*60)

# √
# print(x.shape)
# for layer in model.convLayer:
#     x = layer(x)
#     print(x.shape)
#     print('-'*60)


torch.Size([2])


In [17]:
import torch
x = torch.tensor(
    [[3, 4],
    [5, 12],
    [3, 4]],dtype=torch.float32)
print(x.dtype)

normV = torch.norm(x, dim = 1)
print(normV)
x = x.permute(1, 0)
print(x)
res = torch.div(x, normV)
print(res)
res = res.permute(1, 0)
print(res)
normV2 = torch.norm(res, dim = 1)
print(normV2)

torch.float32
tensor([ 5., 13.,  5.])
tensor([[ 3.,  5.,  3.],
        [ 4., 12.,  4.]])
tensor([[0.6000, 0.3846, 0.6000],
        [0.8000, 0.9231, 0.8000]])
tensor([[0.6000, 0.8000],
        [0.3846, 0.9231],
        [0.6000, 0.8000]])
tensor([1., 1., 1.])


In [65]:
def funcy(x: torch.tensor, k:int):
    return x**(-2*k)

In [85]:
class Pretfm(torch.nn.Module):
    def __init__(self, in_channel) -> None:
        super().__init__()
        self.in_channel = in_channel

    def forward(self, x: torch.tensor):

        y = torch.rand(self.in_channel, 256, 256)

        for i in range(1, self.in_channel+1):
            y[i-1] = funcy(x, i)
        x = y
        return x

In [94]:
in_channel = 3
import torchvision.transforms as T
train_tfm = T.Compose(
    [
        T.Resize((256, 256)),
        # 取逆矩阵 扩充channel
        # Pretfm(in_channel),
        #是否需要数据增强 保留一个问号
        # 层归一化
        # nn.LayerNorm((in_channel, 256, 256))
    ]
)
# t = nn.LayerNorm((in_channel, 256, 256))
# # x = torch.rand(1, 188, 188)
# x = torch.tensor([[[1,1],
#                     [2,2]
#                     ]
#                     ], dtype=torch.float32)
# output = train_tfm(x)
# print(output)
# # print(output.shape)
# print(t(output))

In [95]:
import numpy as np
class Train_set(torch.utils.data.Dataset):

    def __init__(self, dir, id_list, tfm) -> None:
        super().__init__()
        
        self.tensor_list = []
        for id, label in id_list:
            # 在蛋白质数据库文件查找 id.npy
            feature = torch.from_numpy(np.load(dir+id+".npy", allow_pickle=True))
            feature = torch.unsqueeze(feature, 0)
            self.tensor_list.append((feature,
                                        label)
                                        )
        self.tfm = tfm

    def __getitem__(self, idx :int):
        y = self.tensor_list[idx][0]
        y = self.tfm(y)
        label = self.tensor_list[idx][1]
        return y, label

    def __len__(self):
        return len(self.tensor_list)


In [99]:
dir = "../distance_matrix/distance_matrix_inf/"
id_path = "../pair/pair_bool/d1a0aa_.txt"
id_list = []

with open(id_path, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        line = lines.split('\n')[0].split('\t')
        id_list.append((line[0], line[1]))

# print(len(id_list))

dataset = Train_set(dir, id_list[:10], train_tfm)
print(id_list[:10])
print(dataset[0][0].shape)



[('d1a0pa2', '0'), ('d2a0ua1', '0'), ('d3a04a_', '0'), ('d5a0ya1', '0'), ('d5a0ya2', '0'), ('d1a1ia1', '0'), ('d1a1va2', '0'), ('d2a14a1', '0'), ('d2a19a2', '0'), ('d2a1jb1', '0')]
torch.Size([1, 256, 256])


In [100]:
data = np.load("../distance_matrix/distance_matrix_inf/d1a0pa2.npy", allow_pickle=True)
print(data.shape)

(182, 182)


In [ ]:
# 训练过程
#/home/wngys/lab/DeepFold/Code

